## Building classifiers

Here will tune hyperparameters for models.

Starting with XGBoost...

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from Modules import *
sns.set()
%matplotlib inline
import imblearn
import xgboost as xgb

In [2]:
#read in the data; data formated weird with extra column, 
#use only data we need
data = pd.read_csv('Data/Final_trimmed_sequential_data.csv')
df = data.iloc[:, 1:]
y = df['Y']

df.head().T

,0,1,2,3,4
AGE,24.000000,26.000000,34.000000,37.000000,57.000000
Y,1.000000,1.000000,0.000000,0.000000,0.000000
SEX_Female,1.000000,1.000000,1.000000,1.000000,0.000000
SEX_Male,0.000000,0.000000,0.000000,0.000000,1.000000
EDUCATION_Graduate School,0.000000,0.000000,0.000000,0.000000,0.000000
EDUCATION_Other,0.000000,0.000000,0.000000,0.000000,0.000000
EDUCATION_University,1.000000,1.000000,1.000000,1.000000,1.000000
MARRIAGE_Married,1.000000,0.000000,0.000000,1.000000,1.000000
MARRIAGE_Non-married,0.000000,1.000000,1.000000,0.000000,0.000000
PERCENT_OF_LIMIT_BAL1,0.195650,0.022350,0.308011,0.899800,0.132340


In [73]:
# set up the split data
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import sklearn.metrics
from imblearn.over_sampling import SMOTE

#1
X = df.drop(columns = ['Y'])
y = df['Y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2019)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .15, random_state = 2019)

In [74]:
# set up cross-validation purposes

#create  dmatrix
dtrain = xgb.DMatrix(X_train, label = y_train)
dval = xgb.DMatrix(X_val)

train_labels = dtrain.get_label()
ratio = float(np.sum(train_labels == 0)) / np.sum(train_labels == 1)

#create paramters for model
params = {'objective': 'binary:logistic',
          'max_depth': 1,
          'eta': 1,
          'scale_pos_weight':ratio
}
num_rounds = 15

xgb.cv(params, dtrain, num_rounds, metrics=['auc', 'map'])




,train-auc-mean,train-auc-std,train-map-mean,train-map-std,test-auc-mean,test-auc-std,test-map-mean,test-map-std
0,0.643181,0.001377,0.436070,0.002342,0.643168,0.002783,0.432958,0.009323
1,0.693607,0.001310,0.489369,0.004755,0.693564,0.002612,0.483506,0.004628
2,0.710948,0.001968,0.497447,0.005532,0.710914,0.003961,0.499502,0.013214
3,0.724394,0.001320,0.512685,0.005504,0.721749,0.005840,0.512460,0.005119
4,0.737622,0.000906,0.517371,0.007628,0.733006,0.002864,0.515013,0.006016
5,0.741998,0.004748,0.519278,0.007511,0.735499,0.006520,0.513078,0.010684
6,0.747793,0.004430,0.523947,0.004747,0.741270,0.009115,0.523509,0.007536
7,0.750674,0.005949,0.529220,0.007398,0.744426,0.005445,0.523748,0.008467
8,0.754783,0.006020,0.527884,0.004085,0.747076,0.003515,0.525998,0.011590
9,0.758042,0.004055,0.531200,0.002686,0.749112,0.004971,0.524318,0.009692
